In [1]:
import ifcopenshell
import ifcopenshell.util.element as Element
from owlready2 import *
import uuid

# Load IFC file
ifc_file = ifcopenshell.open("Duplex_House.ifc")

#Load Ontology
onto = get_ontology("base_bimonto.owl").load()

quantities_attr = [
    "Length","Width","Height","NetVolume","NetSideArea",
    "GrossVolume","Perimeter","NetAreaGrossArea","OuterSurfaceArea",
    "CrossSectionArea","Span","NumberOfRiser","RiserHeight","TreadLength",
]

In [2]:
objects = ifc_file.by_type("IfcBuildingElement")


for obj in objects :
    obj_type = obj.is_a()
    obj_id = obj.id()
    level = Element.get_container(obj).Name if Element.get_container(obj) else ''
    with onto :
        insta = onto[obj_type](f"{obj_type}_{obj_id}")
        insta.id = obj_id
        insta.type = obj_type
        insta.GlobalId = obj.GlobalId
        insta.Name = obj.Name
        insta.Description = obj.Description
        insta.ObjectType = obj.ObjectType
        insta.Tag = obj.Tag
        insta.PredefinedType = obj.PredefinedType
        
        building_storey = onto["IfcBuildingStorey"](level)
        insta.isLocatedAt = building_storey
        building_storey.containElements.append(insta)
        
        
        Quantitysets = Element.get_psets(obj, qtos_only=True)
        qto_names = list(Quantitysets.keys())
        for qto_name in qto_names:
            set_id = Quantitysets[qto_name]["id"]
            qto_set = onto["IfcQuantitySet"](f"{qto_name}_{set_id}")
            insta.hasQuantitySet.append(qto_set)
            # Iterate over the properties and set them to the instance
            for qto_attr in quantities_attr:
                value = next((value.get(qto_attr) for value in Quantitysets.values() if qto_attr in value), None)
    
                if value is not None:
                    setattr(qto_set, qto_attr, value)
        
        
        property_sets = Element.get_psets(obj, psets_only=True)
        for pset_name, attributes in property_sets.items():
            set_id = property_sets [pset_name]["id"]
            prop_set = onto["IfcPropertySet"](f"{pset_name}_{set_id}")
            insta.hasPropertySet.append(prop_set)
            for attribute, value in attributes.items():
                data_prop = types.new_class(attribute, (DataProperty, FunctionalProperty))
                data_prop.domain = [onto.IfcPropertySet]
                data_prop.range = [type(value)]
                setattr(prop_set, attribute, value)  
  
    
   

In [3]:
onto.save(file= "filled_bimonto.owl", format="rdfxml")

In [56]:
onto.destroy()

In [1]:
from rdflib import Graph

# Load the ontology
g = Graph()
g.parse("filled_bimonto.owl", format="xml")  # Adjust format as needed

g.serialize("filled_bimonto.ttl", format="ttl")

<Graph identifier=N2e132eeb480c4e24ae8408dfd90b1c43 (<class 'rdflib.graph.Graph'>)>